*To run please install the [.NET Interactive Notebooks](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) extension for Visual Studio Code. (When opening the first it will install .NET Interactive)*

# Show data (optional)

In [ ]:
// This may take over 1 mintue
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json" 
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json" 

#r "nuget:Microsoft.ML, 1.7.0"
#r "nuget:Microsoft.ML.AutoML, 0.19.0"
#r "nuget:Microsoft.Data.Analysis, 0.19.0"
#r "nuget:XPlot.Plotly.Interactive, 4.0.6"


Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.19.0 Microsoft.ML, 1.7.0 Microsoft.ML.AutoML, 0.19.0 XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

In [ ]:
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.Data.Analysis;
using XPlot.Plotly;

var data = DataFrame.LoadCsv("data/labeled_data.csv");

var chart = Chart.Plot(
    new     ()
    {
        x = data.Columns["x"],
        y = data.Columns["y"],
        mode = "markers",
        marker = new Marker()
        {
            color = data.Columns["label"],
            colorscale = "Jet"
        }
    }
);

chart.Width = 400;
chart.Height = 400;
chart.Display();


Error: (11,9): error CS0117: 'Trace' does not contain a definition for 'x'
(12,9): error CS0117: 'Trace' does not contain a definition for 'y'
(13,9): error CS0117: 'Trace' does not contain a definition for 'mode'
(14,9): error CS0117: 'Trace' does not contain a definition for 'marker'

# Train

In [ ]:
// One day we can import projects or files
// TODO import NMicorgrad.dll

#r "NMicrograd\bin\Debug\net6.0\NMicrograd.dll"

In [ ]:
// Please see the notebook in the `data` folder for the steps how these were captures
var lines = System.IO.File.ReadAllLines("data\\labeled_data.csv").Skip(1);

// Format is x1, x2, label
var data = lines.Select( l => l.Split(',')) 
   .Select( a => new {
        X = new []{double.Parse(a[0]), double.Parse(a[1])},
        Y = int.Parse(a[2])});

In [ ]:
using NMicrograd;

var model = new MLP(2, new []{16, 16, 1}); // 2-layer neural network

Console.WriteLine(model);
Console.WriteLine($"Number of parameters: {model.GetParameters().Count()}");

MLP of [Layer of [ReLUNeuron(2),ReLUNeuron(2),ReLUNeuron(2),ReLUNeuron(2),ReLUNeuron(2),ReLUNeuron(2),ReLUNeuron(2),ReLUNeuron(2),ReLUNeuron(2),ReLUNeuron(2),ReLUNeuron(2),ReLUNeuron(2),ReLUNeuron(2),ReLUNeuron(2),ReLUNeuron(2),ReLUNeuron(2)],Layer of [ReLUNeuron(16),ReLUNeuron(16),ReLUNeuron(16),ReLUNeuron(16),ReLUNeuron(16),ReLUNeuron(16),ReLUNeuron(16),ReLUNeuron(16),ReLUNeuron(16),ReLUNeuron(16),ReLUNeuron(16),ReLUNeuron(16),ReLUNeuron(16),ReLUNeuron(16),ReLUNeuron(16),ReLUNeuron(16)],Layer of [LinearNeuron(16)]]
Number of parameters: 337


In [ ]:
using System.IO;

// Optional
// Copies exact weights used in the https://github.com/karpathy/micrograd/blob/master/demo.ipynb (from 14 Apr 2020)
// so that the descent is exactly the same
// Please see the notebook in the `data` folder for the steps how these were captures
var weights = File.ReadAllText("data\\weights_data.csv")
    .Split(',')
    .Select( s => double.Parse(s));

if(model.GetParameters().Count() != weights.Count())
{
    throw new Exception($"Bad weights. Has {weights.Count()}, need {model.GetParameters().Count()}");
}

// Fill the weights
foreach((var p, var w) in model.GetParameters().Zip(weights, (p,w) => (p,w)))
    p.Data = w; 

In [ ]:
static (Value TotalLoss, double Accuracy) GetLoss(
    IEnumerable<IEnumerable<double>> x,
    IEnumerable<int> y,
    MLP model)
{
    var inputs = x.Select( d => d.Select( x => new Value(data: x)));

    // forward the model to get scores
    var scores = inputs.Select(input => model.F(input).First());

    // svm "max-margin" loss
    var losses = y.Zip(scores, (yi, scorei) => (1 + -yi*scorei).ReLU());
    
    // var data_loss = new Value(0);
    // foreach(var l in losses) data_loss+=l;
    // data_loss = data_loss * (1.0 / losses.Count());
    var data_loss = losses.Aggregate(new Value(0), (s,d) => s+d) * (1.0 / losses.Count());

    // L2 regularization
    var alpha = 1e-4;
    //var reg_loss = new Value(0);
    //model.GetParameters().Select(p => p*p).ToList().ForEach(p2 => reg_loss += p2); 
    //reg_loss = alpha * reg_loss;
    var reg_loss = alpha * model.GetParameters().Aggregate(new Value(0), (s,d) => s+(d*d));
    var total_loss = data_loss + reg_loss;
    
    // also get accuracy
    var accuracy = y.Zip(scores, (yi, scorei) => (yi > 0) == (scorei.Data > 0));
    return (TotalLoss: total_loss, Accuracy: accuracy!.Sum(a => a ? 1.0 : 0) / accuracy!.Count());
}

In [ ]:
// Actual training
for(int k = 0; k<100; k++)
{
    // forward
    (var total_loss, var acc) = GetLoss(
        x: data.Select(d => d.X), 
        y: data.Select(d => d.Y),
        model);
    
    // backward
    model.ZeroTheGrads();
    total_loss.Backward();
    
    // update (stochastic gradient descent)
    var learning_rate = 1.0 - (0.9*k)/100;
    foreach(var p in model.GetParameters())
        p.Data -= learning_rate * p.Grad;

    if (k % 1 == 0)
        Console.WriteLine($"step {k} loss {string.Format("{0:F17}", total_loss.Data)}, accuracy {acc*100:0.0}%");
}

step 0 loss 0.89584410286832217, accuracy 50.0%
step 1 loss 1.72359053369720128, accuracy 81.0%
step 2 loss 0.74290063138511309, accuracy 77.0%
step 3 loss 0.77056412605842017, accuracy 82.0%
step 4 loss 0.36927933859765377, accuracy 84.0%
step 5 loss 0.31354548191852194, accuracy 86.0%
step 6 loss 0.28142343497724337, accuracy 89.0%
step 7 loss 0.26888733313983904, accuracy 91.0%
step 8 loss 0.25671472860574168, accuracy 91.0%
step 9 loss 0.27048625516379227, accuracy 91.0%
step 10 loss 0.24507023853658050, accuracy 91.0%
step 11 loss 0.25099055297915035, accuracy 92.0%
step 12 loss 0.21560951851922952, accuracy 91.0%
step 13 loss 0.23090378446402726, accuracy 93.0%
step 14 loss 0.20152151227899456, accuracy 92.0%
step 15 loss 0.22574506279282219, accuracy 93.0%
step 16 loss 0.19447987596204114, accuracy 92.0%
step 17 loss 0.21089496199246355, accuracy 93.0%
step 18 loss 0.15983077356303607, accuracy 94.0%
step 19 loss 0.18453748746883916, accuracy 93.0%
step 20 loss 0.189775228560876

In [ ]:
// TODO capture decision boudary at every n steps and create a gif from them